# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
csv_weather_data = pd.read_csv("../WeatherPy/city_weather_data.csv")
csv_weather_data

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,nome,64.5011,-165.4064,25.00,68,1,11.50,US,1619720950
1,port elizabeth,-33.9180,25.5701,59.00,88,0,11.50,ZA,1619721354
2,kavaratti,10.5669,72.6420,85.10,71,35,4.70,IN,1619720978
3,turukhansk,65.8167,87.9833,26.71,87,97,9.17,RU,1619721355
4,sao joao da barra,-21.6403,-41.0511,75.20,69,75,13.80,BR,1619721355
...,...,...,...,...,...,...,...,...,...
572,camacha,33.0833,-16.3333,64.40,59,40,10.36,PT,1619721596
573,yayva,59.3303,57.2641,34.00,80,62,4.54,RU,1619721596
574,paamiut,61.9940,-49.6678,37.17,82,2,5.84,GL,1619721597
575,egvekinot,66.3167,-179.1667,16.86,84,12,5.03,RU,1619721597


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)
locations = csv_weather_data[["Lat", "Lng"]]
humidity = csv_weather_data["Humidity"]
fig = gmaps.figure(center=(20.0,20.0), zoom_level=2)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=np.max(humidity),point_radius=3)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
ideal_weather_data = csv_weather_data.loc[(csv_weather_data["Max Temp"]<80)&
                                             (csv_weather_data["Max Temp"]>70)&
                                             (csv_weather_data["Wind Speed"]<10)&
                                             (csv_weather_data["Cloudiness"]==0)]
hotel_df = ideal_weather_data.dropna().reset_index(drop=True)
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,saint-philippe,-21.3585,55.7679,77.00,83,0,4.61,RE,1619721379
1,chuy,-33.6971,-53.4616,71.01,45,0,3.76,UY,1619720995
2,karratha,-20.7377,116.8463,79.59,40,0,9.15,AU,1619721016
3,san luis,-33.2950,-66.3356,75.25,27,0,9.95,AR,1619721437
4,laranjeiras do sul,-25.4078,-52.4161,73.94,24,0,2.39,BR,1619721450
5,port hedland,-20.3167,118.5667,71.60,64,0,8.05,AU,1619721452
6,saint-pierre,-21.3393,55.4781,77.00,83,0,4.61,RE,1619721096
7,pueblo nuevo,23.3833,-105.3833,73.98,22,0,9.60,MX,1619721470
8,formosa,-26.1775,-58.1781,77.00,47,0,6.91,AR,1619721465
9,pinheiro machado,-31.5783,-53.3811,73.00,54,0,3.00,BR,1619721073


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
except_count = 0

for index,row in hotel_df.iterrows():
    
    params = {
        "location": f"{row[1]},{row[2]}",
        "radius": 5000,
        "type": "lodging",
        "keyword": "hotel",
        "language": "en",
        "key": g_key
    }
    
    try:
        response = requests.get(base_url, params=params).json()
        hotel_name = response["results"][0]["name"]
        hotel_df.at[index,"Hotel Name"] = hotel_name
    except:
        except_count +=1

except_count

2

In [6]:
hotel_df = hotel_df.dropna().reset_index(drop=True)
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,saint-philippe,-21.3585,55.7679,77.00,83,0,4.61,RE,1619721379,Les Embruns Du Baril
1,chuy,-33.6971,-53.4616,71.01,45,0,3.76,UY,1619720995,Hotel Internacional
2,karratha,-20.7377,116.8463,79.59,40,0,9.15,AU,1619721016,Karratha International Hotel
3,san luis,-33.2950,-66.3356,75.25,27,0,9.95,AR,1619721437,GRAN HOTEL SAN LUIS GOLD
4,laranjeiras do sul,-25.4078,-52.4161,73.94,24,0,2.39,BR,1619721450,Laranjeiras Palace Hotel
5,port hedland,-20.3167,118.5667,71.60,64,0,8.05,AU,1619721452,Hedland Hotel
6,saint-pierre,-21.3393,55.4781,77.00,83,0,4.61,RE,1619721096,Hôtel le Saint-Pierre
7,formosa,-26.1775,-58.1781,77.00,47,0,6.91,AR,1619721465,Howard Johnson Hotel And Casino Formosa
8,pinheiro machado,-31.5783,-53.3811,73.00,54,0,3.00,BR,1619721073,Hotel Dal Cortivo (Pinheiro Machado)
9,santiago,-33.4569,-70.6483,75.99,35,0,4.61,CL,1619721555,Solace Hotel Santiago


In [7]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [9]:
# Add marker layer ontop of heat map

markers = gmaps.marker_layer(locations, info_box_content= hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))